<a href="https://colab.research.google.com/github/Hanbin-git/dacon_new_drug/blob/main/kaggle_%EB%8B%A4%EB%A5%B8%20%EC%A0%84%EB%9E%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

biniroun_datafile1_path = kagglehub.dataset_download('biniroun/datafile1')

print('Data source import complete.')


In [ ]:
!pip install pycaret[full] rdkit-pypi --quiet


In [ ]:
import pandas as pd

# ✅ 데이터 경로
data_path = "/kaggle/input/datafile1"

# ✅ CSV 파일 로드
train = pd.read_csv(f"{data_path}/train.csv")
test = pd.read_csv(f"{data_path}/test.csv")
submission = pd.read_csv(f"{data_path}/sample_submission.csv")

# ✅ 데이터 확인
print("✅ train shape:", train.shape)
print("✅ test shape:", test.shape)
print("✅ submission shape:", submission.shape)
train.head()


✅ train shape: (1681, 3)
✅ test shape: (100, 2)
✅ submission shape: (100, 2)


,ID,Canonical_Smiles,Inhibition
0,TRAIN_0000,Cl.OC1(Cc2cccc(Br)c2)CCNCC1,12.50
1,TRAIN_0001,Brc1ccc2OCCc3ccnc1c23,4.45
2,TRAIN_0002,CC1(CO)CC(=NO1)c2cc(c(F)cc2Cl)[N+](=O)[O-],4.92
3,TRAIN_0003,Fc1ccc2nc(Nc3cccc(COc4cccc(c4)C(=O)N5CCOCC5)c3...,71.50
4,TRAIN_0004,CC(C)CC(=O)C1=C(Nc2c(Cl)ccc(Cl)c2C1=O)S(=O)C,18.30


In [ ]:
import os
print(os.listdir("/kaggle/input/datafile1"))

In [ ]:
import os
print(os.path.exists("/kaggle/input/datafile1/train.csv"))  # True면 OK

In [ ]:
# Kaggle 노트북 상단에 추가 (다른 pip install 명령과 함께)
# 기존 pandas를 제거하고 특정 안정 버전으로 설치
!pip uninstall -y pandas
!pip install pandas==1.5.3

# (선택 사항: 다른 라이브러리도 함께 설치해야 한다면)
# !pip install --upgrade joblib loky scikit-learn==1.3.0 rdkit optuna catboost lightgbm xgboost

Found existing installation: pandas 2.1.4
Uninstalling pandas-2.1.4:
  Successfully uninstalled pandas-2.1.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 98.3 MB/s eta 0:00:00:00:01:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
salib 1.5.1 requires pandas>=2.0, but you have pandas 1.5.3 which is incompatible.
dask-cudf-cu12 25.2.2 requires pandas<2.2.4dev0,>=2.0, but you have pandas 1.5.3 which is incompatible.
dask-expr 1.1.21 requires pandas>=2, but you have pandas 1.5.3 which is incompatible.
cudf-cu12 25.2.2 requires pandas<2.2.4dev0,>=2.0, but you have pandas 1.5.3 which is incompatible.
datasets 3.6.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.3.2 which is incompatible.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.11.4 which is incompatible.
woodwork 0.31.0 requires pandas

In [ ]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.feature_selection import VarianceThreshold
from sklearn.linear_model import Ridge
from sklearn.ensemble import StackingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
import os
import warnings
warnings.filterwarnings("ignore")

# ✅ 데이터 경로 설정
data_path = "/kaggle/input/datafile1"

# ✅ 데이터 로드
train_df = pd.read_csv(f"{data_path}/train.csv")
test_df = pd.read_csv(f"{data_path}/test.csv")
submission_df = pd.read_csv(f"{data_path}/sample_submission.csv").copy()  # <- 반드시 copy

# ✅ SMILES → ECFP 변환
def smiles_to_ecfp(smiles, radius=2, n_bits=2048):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return np.zeros(n_bits)
    return np.array(AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits=n_bits))

# ✅ Fingerprint 생성
X_train = np.array([smiles_to_ecfp(s) for s in train_df["Canonical_Smiles"]])
X_test = np.array([smiles_to_ecfp(s) for s in test_df["Canonical_Smiles"]])
y_train = np.log1p(train_df["Inhibition"])  # log1p 변환

# ✅ Feature Selection
selector = VarianceThreshold(threshold=0.01)
X_train_sel = selector.fit_transform(X_train)
X_test_sel = selector.transform(X_test)

# ✅ 모델 정의
base_models = [
    ("lgb", LGBMRegressor(n_estimators=200, learning_rate=0.05, random_state=42)),
    ("xgb", XGBRegressor(n_estimators=200, learning_rate=0.05, random_state=42)),
    ("cat", CatBoostRegressor(iterations=200, learning_rate=0.05, verbose=0, random_state=42)),
]
meta_model = Ridge()

# ✅ 스태킹 앙상블 학습
model = StackingRegressor(estimators=base_models, final_estimator=meta_model, n_jobs=-1)
model.fit(X_train_sel, y_train)

# ✅ 예측 및 역변환
y_pred_log = model.predict(X_test_sel)
y_pred = np.expm1(y_pred_log)  # log1p 역변환

# ✅ 결과 저장 준비
if not isinstance(submission_df, pd.DataFrame):
    raise TypeError("submission_df가 DataFrame이 아닙니다!")

submission_df["Inhibition"] = y_pred

# ✅ 제출 파일 저장
output_path = "/kaggle/working/submission_stack_log1p.csv"
submission_df.to_csv(output_path, index=False)
print(f"✅ 제출 파일 저장 완료: {output_path}")


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.014064 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1542
[LightGBM] [Info] Number of data points in the train set: 1681, number of used features: 771
[LightGBM] [Info] Start training from score 2.996934
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.022376 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1208
[LightGBM] [Info] Number of data points in the train set: 1344, number of used features: 604
[LightGBM] [Info] Start training from score 3.068565
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.017009 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enoug

In [ ]:
# ✅ 라이브러리 로드
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import Descriptors
import xgboost as xgb
import os
import warnings
warnings.filterwarnings("ignore")

# ✅ 데이터 경로 설정 (Kaggle 경로에 맞게)
data_path = "/kaggle/input/datafile1"
output_path = "/kaggle/working/baseline_submit.csv"

# ✅ 데이터 로드
train_df = pd.read_csv(f"{data_path}/train.csv")
test_df = pd.read_csv(f"{data_path}/test.csv")
submission_df = pd.read_csv(f"{data_path}/sample_submission.csv").copy()

# ✅ 분자 특성 추출 함수 정의
def extract_rdkit_features(smiles):
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol is not None:
            return [
                Descriptors.MolWt(mol),             # 분자량
                Descriptors.MolLogP(mol),           # LogP
                Descriptors.NumHAcceptors(mol),     # 수소 수용자 수
                Descriptors.NumHDonors(mol),        # 수소 기부자 수
                Descriptors.TPSA(mol),              # 극성 표면적
                Descriptors.NumRotatableBonds(mol)  # 회전 가능한 결합 수
            ]
    except:
        pass
    return [0.0] * 6  # 실패 시 0으로 채움

# ✅ 훈련 데이터 특성 생성
train_df["Features"] = train_df["Canonical_Smiles"].apply(extract_rdkit_features)
train_x = np.stack(train_df["Features"].values)
train_y = train_df["Inhibition"].astype(float).values

# ✅ 모델 학습
model = xgb.XGBRegressor(random_state=42, n_estimators=300, learning_rate=0.05)
model.fit(train_x, train_y)

# ✅ 테스트 데이터 특성 생성
test_df["Features"] = test_df["Canonical_Smiles"].apply(extract_rdkit_features)
test_x = np.stack(test_df["Features"].values)

# ✅ 예측 수행
test_y_pred = model.predict(test_x)

# ✅ 제출 파일 생성
submission_df["Inhibition"] = test_y_pred
submission_df.to_csv(output_path, index=False)
print(f"✅ 제출 파일 저장 완료: {output_path}")


✅ 제출 파일 저장 완료: /kaggle/working/baseline_submit.csv


In [ ]:
# ✅ 라이브러리 로드
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import Descriptors
import xgboost as xgb
import os
import warnings
warnings.filterwarnings("ignore")

# ✅ 데이터 경로 설정
data_path = "/kaggle/input/datafile1"
output_path = "/kaggle/working/baseline_submit2.csv"

# ✅ 데이터 로드
train_df = pd.read_csv(f"{data_path}/train.csv")
test_df = pd.read_csv(f"{data_path}/test.csv")
submission_df = pd.read_csv(f"{data_path}/sample_submission.csv").copy()  # <- 반드시 copy

# ✅ 분자 특성 추출 함수 정의
def extract_rdkit_features(smiles):
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol is not None:
            return [
                Descriptors.MolWt(mol),             # 분자량
                Descriptors.MolLogP(mol),           # LogP
                Descriptors.NumHAcceptors(mol),     # 수소 수용자 수
                Descriptors.NumHDonors(mol),        # 수소 기부자 수
                Descriptors.TPSA(mol),              # 극성 표면적
                Descriptors.NumRotatableBonds(mol)  # 회전 가능한 결합 수
            ]
    except:
        pass
    return [0.0] * 6  # 실패 시 0으로 채움

# ✅ 훈련 데이터 특성 생성
train_df["Features"] = train_df["Canonical_Smiles"].apply(extract_rdkit_features)
train_x = np.stack(train_df["Features"].values)
train_y = train_df["Inhibition"].astype(float).values

# ✅ 모델 학습
model = xgb.XGBRegressor(random_state=42, n_estimators=300, learning_rate=0.05)
model.fit(train_x, train_y)

# ✅ 테스트 데이터 특성 생성
test_df["Features"] = test_df["Canonical_Smiles"].apply(extract_rdkit_features)
test_x = np.stack(test_df["Features"].values)

# ✅ 예측 수행
test_y_pred = model.predict(test_x)

# ✅ 제출 파일 생성
submission_df["Inhibition"] = test_y_pred
submission_df.to_csv(output_path, index=False)
print(f"✅ 제출 파일 저장 완료: {output_path}")


✅ 제출 파일 저장 완료: /kaggle/working/baseline_submit2.csv


In [ ]:
# ==============================================================================
# --- Kaggle 최적화 신약개발 예측 파이프라인 (PyCaret 기반) ---
# ==============================================================================

# ✅ 1. 라이브러리 임포트
import pandas as pd
import numpy as np
import io # 예제 데이터 생성 시 필요 (현재는 사용하지 않음)
import warnings
import time

# RDKit 및 PyCaret 라이브러리 임포트
from rdkit import Chem
from rdkit.Chem import AllChem
from pycaret.regression import setup, compare_models, finalize_model, predict_model, pull

# 경고 메시지 무시 및 RDKit 에러 메시지 억제
warnings.filterwarnings(action='ignore')
from rdkit import rdBase
rdBase.DisableLog('rdApp.error')

print(f"--- Kaggle 파이프라인 시작: {time.strftime('%Y-%m-%d %H:%M:%S', time.localtime())} ---")

# ✅ 2. 데이터 로드 (Kaggle 전용 경로)
# Kaggle 데이터셋 경로를 사용합니다. 'datafile1'은 실제 데이터셋 이름에 따라 변경해야 할 수 있습니다.
# 예를 들어, 대회 데이터셋 이름이 'dacon-cyp3a4-inhibition-prediction'이라면
# data_path = "/kaggle/input/dacon-cyp3a4-inhibition-prediction" 으로 변경합니다.
data_path = "/kaggle/input/datafile1" # 예시 경로

try:
    train_df = pd.read_csv(f'{data_path}/train.csv')
    test_df = pd.read_csv(f'{data_path}/test.csv')
    submission_df = pd.read_csv(f'{data_path}/sample_submission.csv').copy() # .copy() 추가
    print("파일에서 데이터를 성공적으로 로드했습니다.")
except FileNotFoundError:
    print(f"오류: 파일을 찾을 수 없습니다. 경로를 확인해주세요: {data_path}")
    # Kaggle 환경에서는 FileNotFoundError 발생 시 실행이 중단되므로, 예외 처리는 주로 개발 환경에서 유용합니다.
    # Kaggle에서는 데이터셋이 제대로 추가되었는지 확인해야 합니다.
    raise # Kaggle에서는 이 시점에서 오류를 발생시켜 데이터셋 문제를 알립니다.


# --- 1. 특성 추출 (Feature Engineering) - Morgan Fingerprints ---
# 모든 RDKit Descriptors를 포함하는 것이 성능에 유리할 수 있습니다.
# 이전 대화에서 제공했던 'featurize_all_rdkit' 함수를 사용하여 모든 Descriptors와 Fingerprint를 함께 사용하는 것을 고려하세요.
# 현재 코드는 제공해주신 코드와 동일하게 Morgan Fingerprint만 사용합니다.
def get_morgan_fingerprint(smiles_string, n_bits=2048, radius=2):
    """SMILES 문자열을 Morgan Fingerprint(numpy array)로 변환"""
    try:
        mol = Chem.MolFromSmiles(smiles_string)
        if mol is not None:
            fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits=n_bits)
            return np.array(fp, dtype=int) # PyCaret에 맞게 int 타입 유지
        else:
            return np.zeros(n_bits, dtype=int)
    except:
        return np.zeros(n_bits, dtype=int)

print("\n특성 추출을 시작합니다 (Morgan Fingerprints)...")
# 학습 데이터와 테스트 데이터에 함수 적용
train_fps = train_df['Canonical_Smiles'].apply(get_morgan_fingerprint)
test_fps = test_df['Canonical_Smiles'].apply(get_morgan_fingerprint)

# --- PyCaret을 위한 데이터 준비 ---
# Fingerprint를 DataFrame으로 변환
fp_columns = [f'FP_{i}' for i in range(2048)]
train_fp_df = pd.DataFrame(np.vstack(train_fps.values), columns=fp_columns)
test_fp_df = pd.DataFrame(np.vstack(test_fps.values), columns=fp_columns)

# PyCaret 학습을 위해 특성과 타겟 변수를 하나의 DataFrame으로 결합
# Inhibition은 PyCaret 내부에서 자동으로 처리되므로 log1p 변환은 PyCaret 외부에서 수행하지 않습니다.
# 만약 PyCaret 내부에서 log1p 변환을 원한다면, setup() 함수의 transformation=True, transformation_method='yeo-johnson' 등을 고려할 수 있습니다.
pycaret_train_df = pd.concat([train_fp_df, train_df['Inhibition']], axis=1)

print(f"PyCaret에 사용할 학습 데이터 형태: {pycaret_train_df.shape}")
print(f"PyCaret에 사용할 테스트 데이터 형태: {test_fp_df.shape}")
print("특성 추출 완료.")

# --- 2. PyCaret을 이용한 모델 학습 및 예측 ---
print("\nPyCaret 환경 설정을 시작합니다...")
# PyCaret 회귀 환경 설정
# session_id는 재현성을 위해 고정합니다.
# fold=5로 설정하여 5-Fold 교차 검증을 수행합니다.
# normalize=True: MinMax 스케일링은 FP에 덜 중요할 수 있지만, Descriptors와 함께 사용 시 필요
s = setup(data=pycaret_train_df,
          target='Inhibition',
          session_id=42,
          fold=5, # KFold(n_splits=5)와 동일
          normalize=True,
          normalize_method='minmax', # MinMax 또는 zscore
          n_jobs=-1, # Kaggle에서 모든 CPU 코어 사용
          verbose=False) # 상세 정보 출력 생략 (필요시 True로 변경하여 디버깅)

print("PyCaret 환경 설정 완료.")

print("\n다양한 모델을 비교하여 최적의 모델을 찾습니다...")
# 여러 모델을 비교하고 RMSE 기준으로 가장 좋은 모델을 선택
# include 파라미터로 특정 모델만 비교할 수 있습니다 (예: include=['lightgbm', 'xgboost', 'catboost'])
best_model = compare_models(sort='RMSE', n_select=1, verbose=True) # 진행 과정 출력 확인

print("\n모델 비교 결과 (상위 5개):")
# pull() 함수를 사용하여 비교 결과를 DataFrame으로 가져와 출력
results = pull()
print(results.head())

print(f"\n최적 모델로 '{results.index[0]}'가 선택되었습니다.")

# --- 3. 최종 모델 학습 및 예측 ---
print("\n최종 모델을 전체 데이터로 학습합니다...")
# 선택된 최적의 모델을 전체 학습 데이터로 다시 학습시킵니다.
final_model = finalize_model(best_model)
print("최종 모델 학습 완료.")

print("\n테스트 데이터에 대한 예측을 수행합니다...")
# 최종 모델로 테스트 데이터 예측
predictions = predict_model(final_model, data=test_fp_df)
print("예측 완료.")

# --- 4. 제출 파일 생성 ---
print("\n제출 파일을 생성합니다...")
# 예측 결과가 0~100 범위를 벗어나지 않도록 클리핑
# PyCaret 3.x 버전에서는 예측 결과 컬럼명이 'prediction_label' 입니다.
final_predictions = np.clip(predictions['prediction_label'].values, 0, 100)

submission_df['Inhibition'] = final_predictions

# ✅ 5. 제출 파일 저장 (Kaggle 전용 경로)
output_path = "/kaggle/working/submission_pycaret.csv"
submission_df.to_csv(output_path, index=False)

print(f"✅ 제출 파일 저장 완료: {output_path}")
print("\n최종 제출 파일 미리보기:")
print(submission_df.head())

print(f"\n--- Kaggle 파이프라인 완료: {time.strftime('%Y-%m-%d %H:%M:%S', time.localtime())} ---")

--- Kaggle 파이프라인 시작: 2025-06-27 08:56:53 ---
파일에서 데이터를 성공적으로 로드했습니다.

특성 추출을 시작합니다 (Morgan Fingerprints)...
PyCaret에 사용할 학습 데이터 형태: (1681, 2049)
PyCaret에 사용할 테스트 데이터 형태: (100, 2048)
특성 추출 완료.

PyCaret 환경 설정을 시작합니다...
PyCaret 환경 설정 완료.

다양한 모델을 비교하여 최적의 모델을 찾습니다...


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
br,Bayesian Ridge,21.0313,643.1184,25.3313,0.0841,1.3562,5.5619,0.9240
rf,Random Forest Regressor,21.0550,661.6154,25.6895,0.0580,1.3258,5.1628,6.2240
llar,Lasso Least Angle Regression,21.8003,678.5771,26.0170,0.0342,1.3839,5.8185,0.1440
lasso,Lasso Regression,21.8003,678.5772,26.0170,0.0342,1.3839,5.8186,0.1700
en,Elastic Net,21.8877,682.6070,26.0949,0.0284,1.3875,5.8932,0.1420
gbr,Gradient Boosting Regressor,21.5632,684.5222,26.1268,0.0261,1.3760,5.7961,1.1220
catboost,CatBoost Regressor,21.5711,694.4035,26.3328,0.0081,1.3714,5.6998,6.3020
lightgbm,Light Gradient Boosting Machine,21.5410,695.4474,26.3534,0.0074,1.3694,5.2157,32.4780
ada,AdaBoost Regressor,22.3830,695.5039,26.3583,0.0069,1.4323,6.5755,1.0860
dummy,Dummy Regressor,22.2970,708.6996,26.5932,-0.0094,1.4002,6.0433,0.1220



모델 비교 결과 (상위 5개):
                              Model      MAE       MSE     RMSE      R2  \
br                   Bayesian Ridge  21.0313  643.1184  25.3313  0.0841   
rf          Random Forest Regressor  21.0550  661.6154  25.6895  0.0580   
llar   Lasso Least Angle Regression  21.8003  678.5771  26.0170  0.0342   
lasso              Lasso Regression  21.8003  678.5772  26.0170  0.0342   
en                      Elastic Net  21.8877  682.6070  26.0949  0.0284   

        RMSLE    MAPE  TT (Sec)  
br     1.3562  5.5619     0.924  
rf     1.3258  5.1628     6.224  
llar   1.3839  5.8185     0.144  
lasso  1.3839  5.8186     0.170  
en     1.3875  5.8932     0.142  

최적 모델로 'br'가 선택되었습니다.

최종 모델을 전체 데이터로 학습합니다...
최종 모델 학습 완료.

테스트 데이터에 대한 예측을 수행합니다...


예측 완료.

제출 파일을 생성합니다...
✅ 제출 파일 저장 완료: /kaggle/working/submission_pycaret.csv

최종 제출 파일 미리보기:
         ID  Inhibition
0  TEST_000   34.716853
1  TEST_001   29.709531
2  TEST_002   25.155910
3  TEST_003   38.779637
4  TEST_004   26.630452

--- Kaggle 파이프라인 완료: 2025-06-27 09:02:52 ---


In [ ]:
# # ✅ 1. 라이브러리
# import pandas as pd
# import numpy as np
# from rdkit import Chem
# from rdkit.Chem import AllChem
# from lightgbm import LGBMRegressor
# from sklearn.model_selection import KFold
# from sklearn.metrics import mean_absolute_error
# from tqdm import tqdm

# # ✅ 2. 데이터 로드
# data_path = "/kaggle/input/datafile1"
# train = pd.read_csv(f"{data_path}/train.csv")
# test = pd.read_csv(f"{data_path}/test.csv")
# submission = pd.read_csv(f"{data_path}/sample_submission.csv")

# # ✅ 3. Fingerprint 변환 함수
# def smiles_to_morgan(smiles_list, radius=2, n_bits=2048):
#     fps = []
#     for smi in tqdm(smiles_list):
#         mol = Chem.MolFromSmiles(smi)
#         if mol is not None:
#             fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits=n_bits)
#             arr = np.zeros((1,), dtype=int)
#             AllChem.DataStructs.ConvertToNumpyArray(fp, arr)
#             fps.append(arr)
#         else:
#             fps.append(np.zeros((n_bits,), dtype=int))  # 실패한 경우 0벡터
#     return np.array(fps)

# # ✅ 4. Feature Engineering
# X = smiles_to_morgan(train["Canonical_Smiles"])
# X_test = smiles_to_morgan(test["Canonical_Smiles"])
# y = train["Inhibition"]

# # ✅ 5. 모델 훈련 (LightGBM + 5-Fold CV)
# preds = np.zeros(X_test.shape[0])
# kf = KFold(n_splits=5, shuffle=True, random_state=42)
# mae_scores = []

# for train_idx, val_idx in kf.split(X):
#     X_tr, X_val = X[train_idx], X[val_idx]
#     y_tr, y_val = y.iloc[train_idx], y.iloc[val_idx]

#     model = LGBMRegressor(random_state=42)
#     model.fit(X_tr, y_tr)

#     val_pred = model.predict(X_val)
#     mae = mean_absolute_error(y_val, val_pred)
#     mae_scores.append(mae)

#     preds += model.predict(X_test) / kf.n_splits

# print("✅ 평균 MAE:", np.mean(mae_scores))

# # ✅ 6. 제출 파일 생성
# submission["Inhibition"] = preds
# submission.to_csv("submission.csv", index=False)
# print("🎉 submission.csv 저장 완료!")
# #

In [ ]:
import pandas as pd
import io
import warnings
from rdkit import Chem
from rdkit.Chem import AllChem
from pycaret.regression import *
from rdkit import rdBase
rdBase.DisableLog('rdApp.error')
warnings.filterwarnings('ignore')

# ✅ 실제 Kaggle 경로 사용
train_path = "/kaggle/input/datafile1/train.csv"
test_path = "/kaggle/input/datafile1/test.csv"
submission_path = "/kaggle/input/datafile1/sample_submission.csv"

train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)
submission_df = pd.read_csv(submission_path)

print("✅ train_df:", train_df.shape)
print("✅ test_df:", test_df.shape)
print("✅ submission_df:", submission_df.shape)


In [ ]:
import numpy as np

def get_morgan_fingerprint(smiles_string, n_bits=2048, radius=2):
    """SMILES → Morgan Fingerprint (numpy array)"""
    try:
        mol = Chem.MolFromSmiles(smiles_string)
        if mol is not None:
            fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits=n_bits)
            return np.array(fp)
        else:
            return np.zeros(n_bits, dtype=int)
    except:
        return np.zeros(n_bits, dtype=int)

# Fingerprint 추출
train_fps = train_df['Canonical_Smiles'].apply(get_morgan_fingerprint)
test_fps = test_df['Canonical_Smiles'].apply(get_morgan_fingerprint)

# DataFrame 변환
fp_columns = [f'FP_{i}' for i in range(2048)]
train_fp_df = pd.DataFrame(np.vstack(train_fps.values), columns=fp_columns)
test_fp_df = pd.DataFrame(np.vstack(test_fps.values), columns=fp_columns)

# PyCaret용 최종 학습 데이터
pycaret_train_df = pd.concat([train_fp_df, train_df['Inhibition']], axis=1)


In [ ]:
# ✅ PyCaret 환경 설정
s = setup(data=pycaret_train_df,
          target='Inhibition',
          session_id=42,
          fold=5,
          normalize=True,
          normalize_method='minmax',
          verbose=False)

# ✅ 여러 모델 비교 및 최적 모델 선택
best_model = compare_models(sort='RMSE', n_select=1)


In [ ]:
# ✅ 최종 모델 학습
final_model = finalize_model(best_model)

# ✅ 테스트 데이터 예측
predictions = predict_model(final_model, data=test_fp_df)

# ✅ 예측값 0~100 범위로 클리핑
final_predictions = np.clip(predictions['prediction_label'].values, 0, 100)

# ✅ 제출 파일 생성
submission_df['Inhibition'] = final_predictions
submission_df.to_csv("/kaggle/working/submission_pycaret.csv", index=False)

print("✅ 제출 파일 생성 완료 → submission_pycaret.csv")
submission_df.head()


In [ ]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors
from pycaret.regression import setup, compare_models, finalize_model, predict_model
import warnings
from rdkit import rdBase

warnings.filterwarnings('ignore')
rdBase.DisableLog('rdApp.error') # RDKit 에러 메시지 억제

# ✅ 실제 Kaggle 경로 사용
train_path = "/kaggle/input/datafile1/train.csv"
test_path = "/kaggle/input/datafile1/test.csv"
submission_path = "/kaggle/input/datafile1/sample_submission.csv"

train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)
submission_df = pd.read_csv(submission_path)

# --- 개선된 피처 추출 함수 (RDKit Descriptors + Morgan Fingerprints) ---
def get_all_features(smiles_string, n_bits=2048, radius=2):
    mol = Chem.MolFromSmiles(str(smiles_string))
    if mol is None:
        num_descriptors = len(Descriptors._descList)
        return np.zeros(num_descriptors + n_bits, dtype=float)

    descriptor_values = []
    for desc_name, desc_func in Descriptors._descList:
        try:
            val = desc_func(mol)
            if np.isnan(val) or np.isinf(val):
                descriptor_values.append(0.0) # NaN 또는 Inf는 0으로 처리
            else:
                descriptor_values.append(val)
        except:
            descriptor_values.append(0.0) # 계산 오류 시 0으로 처리

    try:
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits=n_bits)
        morgan_fp_values = np.array(fp, dtype=float)
    except:
        morgan_fp_values = np.zeros(n_bits, dtype=float) # 지문 계산 오류 시 0으로 처리

    return np.concatenate([descriptor_values, morgan_fp_values])

print("피처 추출 시작 (RDKit Descriptors + Morgan Fingerprints)...")

# 모든 SMILES에 대해 피처 추출
train_features_raw = np.vstack(train_df['Canonical_Smiles'].apply(get_all_features).values)
test_features_raw = np.vstack(test_df['Canonical_Smiles'].apply(get_all_features).values)

# 컬럼 이름 생성 (Descriptors + Fingerprints)
descriptor_names = [desc[0] for desc in Descriptors._descList]
fp_columns = [f'FP_{i}' for i in range(2048)]
all_feature_columns = descriptor_names + fp_columns

train_features_df = pd.DataFrame(train_features_raw, columns=all_feature_columns)
test_features_df = pd.DataFrame(test_features_raw, columns=all_feature_columns)

print(f"훈련 데이터 피처 형태: {train_features_df.shape}")
print(f"테스트 데이터 피처 형태: {test_features_df.shape}")

# PyCaret용 최종 학습 데이터
pycaret_train_df = pd.concat([train_features_df, train_df['Inhibition']], axis=1)

# ✅ PyCaret 환경 설정 (normalize=True 유지, minmax 또는 zscore 고려)
s = setup(data=pycaret_train_df,
          target='Inhibition',
          session_id=42,
          fold=5,
          normalize=True,
          normalize_method='zscore', # 'minmax' 또는 'zscore' 시도
          transformation=False, # 데이터 분포가 심하게 왜곡되지 않았다면 우선 False
          n_jobs=-1, # CPU 코어 전체 사용
          verbose=False)

# ✅ 여러 모델 비교 및 최적 모델 선택
# 앙상블 모델(LGBM, XGBoost, CatBoost)이 좋은 성능을 보일 수 있습니다.
best_model = compare_models(sort='RMSE', n_select=1, include=['lightgbm', 'xgboost', 'catboost', 'et'])
# 'et' (Extra Trees Regressor)는 무작위성을 높여 과적합을 줄이는 데 도움이 될 수 있습니다.

# ✅ 최종 모델 학습
final_model = finalize_model(best_model)

# ✅ 테스트 데이터 예측
predictions = predict_model(final_model, data=test_features_df)

# ✅ 예측값 0~100 범위로 클리핑
final_predictions = np.clip(predictions['prediction_label'].values, 0, 100)

# ✅ 제출 파일 생성
submission_df['Inhibition'] = final_predictions
submission_df.to_csv("/kaggle/working/submission_pycaret_enhanced_features.csv", index=False)

print("✅ 제출 파일 생성 완료 → submission_pycaret_enhanced_features.csv")
submission_df.head()

In [ ]:
# ✅ 1. 필수 라이브러리
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
import optuna
import warnings
warnings.filterwarnings("ignore")

# ✅ 2. 경로 설정
train_path = "/kaggle/input/datafile1/train.csv"
test_path = "/kaggle/input/datafile1/test.csv"
submission_path = "/kaggle/input/datafile1/sample_submission.csv"

train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)
submission_df = pd.read_csv(submission_path)

# ✅ 3. 피처 추출 함수 (Descriptors + Morgan Fingerprint)
def get_all_features(smiles_string, n_bits=2048, radius=2):
    mol = Chem.MolFromSmiles(str(smiles_string))
    if mol is None:
        num_descriptors = len(Descriptors._descList)
        return np.zeros(num_descriptors + n_bits, dtype=float)

    descriptor_values = []
    for desc_name, desc_func in Descriptors._descList:
        try:
            val = desc_func(mol)
            if np.isnan(val) or np.isinf(val):
                descriptor_values.append(0.0)
            else:
                descriptor_values.append(val)
        except:
            descriptor_values.append(0.0)

    try:
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits=n_bits)
        morgan_fp_values = np.array(fp, dtype=float)
    except:
        morgan_fp_values = np.zeros(n_bits, dtype=float)

    return np.concatenate([descriptor_values, morgan_fp_values])

# ✅ 4. 피처 생성
X_train = np.vstack(train_df['Canonical_Smiles'].apply(get_all_features).values)
X_test = np.vstack(test_df['Canonical_Smiles'].apply(get_all_features).values)
y_train = train_df['Inhibition'].values

# ✅ 5. Optuna 튜닝 함수 정의
from lightgbm import early_stopping, log_evaluation

def objective(trial):
    params = {
        'objective': 'regression',
        'metric': 'rmse',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'n_estimators': 10000,
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-3, 0.1),
        'num_leaves': trial.suggest_int('num_leaves', 20, 128),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'subsample': trial.suggest_uniform('subsample', 0.4, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.4, 1.0),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 10),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-3, 10),
    }

    cv = KFold(n_splits=5, shuffle=True, random_state=42)
    rmse_list = []

    for train_idx, valid_idx in cv.split(X_train):
        X_tr, X_val = X_train[train_idx], X_train[valid_idx]
        y_tr, y_val = y_train[train_idx], y_train[valid_idx]

        model = lgb.LGBMRegressor(**params)
        model.fit(
            X_tr, y_tr,
            eval_set=[(X_val, y_val)],
            callbacks=[early_stopping(50, verbose=False), log_evaluation(0)]
        )

        preds = model.predict(X_val)
        rmse = mean_squared_error(y_val, preds, squared=False)
        rmse_list.append(rmse)

    return np.mean(rmse_list)

# ✅ 6. Optuna 최적화 실행
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=30)

# ✅ 7. 최적 파라미터로 최종 학습
best_params = study.best_params
best_params.update({
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'verbosity': -1,
    'n_estimators': 10000
})

final_model = lgb.LGBMRegressor(**best_params)
final_model.fit(X_train, y_train)

# ✅ 8. 테스트 예측 및 제출
preds_test = final_model.predict(X_test)
submission_df['Inhibition'] = np.clip(preds_test, 0, 100)
submission_df.to_csv("/kaggle/working/submission_lgb_optuna.csv", index=False)


In [ ]:
# ✅ 1. 라이브러리
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import RidgeCV
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
from rdkit import Chem
from rdkit.Chem import Descriptors

# ✅ 2. 데이터 로딩
train = pd.read_csv("/kaggle/input/datafile1/train.csv")
test = pd.read_csv("/kaggle/input/datafile1/test.csv")
submission = pd.read_csv("/kaggle/input/datafile1/sample_submission.csv")

# ✅ 3. Feature Engineering (중요한 분자 feature 추출)
def featurize(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return [0]*6
    return [
        Descriptors.MolWt(mol),         # 분자량
        Descriptors.MolLogP(mol),       # 지용성
        Descriptors.NumHDonors(mol),    # 수소 결합 donor 수
        Descriptors.NumHAcceptors(mol), # 수소 결합 acceptor 수
        Descriptors.TPSA(mol),          # 극성 표면적
        Descriptors.NumRotatableBonds(mol) # 회전 가능한 결합 수
    ]

feature_names = ['MolWt', 'MolLogP', 'NumHDonors', 'NumHAcceptors', 'TPSA', 'RotatableBonds']

train_feats = train['Canonical_Smiles'].apply(featurize)
test_feats = test['Canonical_Smiles'].apply(featurize)

X = pd.DataFrame(train_feats.tolist(), columns=feature_names)
X_test = pd.DataFrame(test_feats.tolist(), columns=feature_names)
y = np.log1p(train['Inhibition'])  # ✅ log1p 변환

# ✅ 4. 모델 정의 (Stacking)
base_models = [
    ('xgb', XGBRegressor(n_estimators=100, max_depth=3, learning_rate=0.1, random_state=42)),
    ('lgbm', LGBMRegressor(n_estimators=100, max_depth=3, learning_rate=0.1, random_state=42)),
    ('cat', CatBoostRegressor(verbose=0, n_estimators=100, learning_rate=0.1, depth=3, random_state=42))
]

meta_model = RidgeCV()

model = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    cv=5,
    n_jobs=-1
)

# ✅ 5. 학습
model.fit(X, y)

# ✅ 6. 예측 및 역변환
preds = model.predict(X_test)
submission['Inhibition'] = np.clip(np.expm1(preds), 0, 100)
submission.to_csv("submission_stacking_log1p_feats.csv", index=False)


In [ ]:
# import pandas as pd
# import numpy as np
# from sklearn.model_selection import KFold
# from sklearn.preprocessing import StandardScaler
# from sklearn.pipeline import Pipeline
# from sklearn.ensemble import StackingRegressor
# from sklearn.linear_model import RidgeCV
# from xgboost import XGBRegressor
# from lightgbm import LGBMRegressor
# from catboost import CatBoostRegressor
# from sklearn.metrics import mean_squared_error, mean_absolute_error # MAE도 함께 확인 권장
# from rdkit import Chem
# from rdkit.Chem import Descriptors, AllChem # AllChem 임포트
# import warnings
# from rdkit import rdBase

# warnings.filterwarnings('ignore')
# rdBase.DisableLog('rdApp.error') # RDKit 에러 메시지 억제

# # ✅ 2. 데이터 로딩
# train = pd.read_csv("/kaggle/input/datafile1/train.csv")
# test = pd.read_csv("/kaggle/input/datafile1/test.csv")
# submission = pd.read_csv("/kaggle/input/datafile1/sample_submission.csv")

# # ✅ 3. Feature Engineering (RDKit Descriptors + Morgan Fingerprints로 변경)
# def featurize_all(smiles_string, n_bits=2048, radius=2):
#     mol = Chem.MolFromSmiles(str(smiles_string))
#     if mol is None:
#         # 분자 생성 실패 시 모든 피처를 0으로 채움
#         num_descriptors = len(Descriptors._descList)
#         return np.zeros(num_descriptors + n_bits, dtype=float)

#     descriptor_values = []
#     for desc_name, desc_func in Descriptors._descList:
#         try:
#             val = desc_func(mol)
#             if np.isnan(val) or np.isinf(val):
#                 descriptor_values.append(0.0) # NaN 또는 Inf는 0으로 처리
#             else:
#                 descriptor_values.append(val)
#         except:
#             descriptor_values.append(0.0) # 계산 오류 시 0으로 처리

#     try:
#         fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits=n_bits)
#         morgan_fp_values = np.array(fp, dtype=float)
#     except:
#         morgan_fp_values = np.zeros(n_bits, dtype=float) # 지문 계산 오류 시 0으로 처리

#     return np.concatenate([descriptor_values, morgan_fp_values])

# print("피처 추출 시작 (RDKit Descriptors + Morgan Fingerprints)...")

# train_features_raw = train['Canonical_Smiles'].apply(featurize_all)
# test_features_raw = test['Canonical_Smiles'].apply(featurize_all)

# # DataFrame 변환
# # 컬럼 이름 생성
# descriptor_names = [desc[0] for desc in Descriptors._descList]
# fp_columns = [f'FP_{i}' for i in range(2048)]
# feature_names_all = descriptor_names + fp_columns

# X = pd.DataFrame(train_features_raw.tolist(), columns=feature_names_all)
# X_test = pd.DataFrame(test_features_raw.tolist(), columns=feature_names_all)
# y = train['Inhibition'] # ✅ log1p 변환은 Stacking 모델에 따라 선택적으로 적용

# print(f"훈련 데이터 피처 형태: {X.shape}")
# print(f"테스트 데이터 피처 형태: {X_test.shape}")


# # ✅ 4. 모델 정의 (Stacking) - StandardScaler 파이프라인에 추가
# # 피처 수가 많아지면 스케일링이 중요해집니다.
# base_models = [
#     ('xgb', XGBRegressor(n_estimators=300, max_depth=5, learning_rate=0.05, random_state=42, n_jobs=-1)),
#     ('lgbm', LGBMRegressor(n_estimators=300, max_depth=5, learning_rate=0.05, random_state=42, n_jobs=-1, verbose=-1)),
#     ('cat', CatBoostRegressor(verbose=0, n_estimators=300, learning_rate=0.05, depth=6, random_state=42))
# ]

# meta_model = RidgeCV(cv=5) # RidgeCV도 내부적으로 CV를 수행합니다.

# # Pipeline을 사용하여 스케일러와 스태킹 모델을 연결
# # StackingRegressor는 내부적으로 자체 CV를 수행하므로, 파이프라인의 CV와 혼동하지 않도록 주의.
# # 여기서는 피처 스케일링을 파이프라인에 포함시키는 것이 목적.
# pipeline_model = Pipeline([
#     ('scaler', StandardScaler()), # 데이터 스케일링 추가
#     ('stacking', StackingRegressor(
#         estimators=base_models,
#         final_estimator=meta_model,
#         cv=KFold(n_splits=5, shuffle=True, random_state=42), # 명시적으로 KFold 사용 (GroupKFold 문제 우회)
#         n_jobs=-1,
#         passthrough=True # 기본 모델의 예측 외에 원본 피처도 meta_model에 전달
#     ))
# ])

# # ✅ 5. 학습
# print("\n모델 학습 시작...")
# pipeline_model.fit(X, y) # y는 이미 스케일링되지 않은 원래 값을 사용

# # ✅ 6. 예측 및 역변환
# print("예측 수행 및 제출 파일 생성...")
# preds = pipeline_model.predict(X_test)
# submission['Inhibition'] = np.clip(preds, 0, 100) # 예측값 클리핑 (log1p를 사용하지 않았으므로 expm1 불필요)
# submission.to_csv("submission_stacking_enhanced_feats.csv", index=False)

# print("\n제출 파일 생성 완료 → submission_stacking_enhanced_feats.csv")
# print(submission.head())

# # (선택 사항) 훈련 데이터에 대한 성능 확인
# train_preds = pipeline_model.predict(X)
# train_mae = mean_absolute_error(y, np.clip(train_preds, 0, 100))
# train_rmse = np.sqrt(mean_squared_error(y, np.clip(train_preds, 0, 100)))
# print(f"훈련 데이터 MAE: {train_mae:.4f}")
# print(f"훈련 데이터 RMSE: {train_rmse:.4f}")